In [9]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *
from BayesNet import *

In [16]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

Vertices:
 V(0, in:1, out:2)
 V(1, in:2, out:1)
 V(2, in:3, out:0)
 V(3, in:0, out:4)
 V(4, in:2, out:1)
 V(5, in:1, out:0)
 V(6, in:0, out:3)
 V(7, in:3, out:0)
 V(8, in:2, out:2)
 V(9, in:0, out:1)
------------------------------------------------------
Edges:
 E(0, 1, 0)
 E(0, 5, 0)
 E(1, 7, 0)
 E(3, 2, 0)
 E(3, 4, 0)
 E(3, 7, 0)
 E(3, 8, 0)
 E(4, 8, 0)
 E(6, 0, 0)
 E(6, 1, 0)
 E(6, 2, 0)
 E(8, 2, 0)
 E(8, 7, 0)
 E(9, 4, 0)



In [24]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

topoBFS: ['3', '6', '9', '0', '4', '1', '5', '8', '2', '7']
topoDFS: ['3', '6', '0', '5', '1', '9', '4', '8', '2', '7']


In [8]:
import numpy as np

In [1514]:
class ConditionalProbabilityTable():
    def __init__(self,initTable,initParents,initNodeName):
        self.ProbabilityTable = np.array(initTable).T.astype(dtype='<U21')  
        self.ParentsList = initParents
        self.NodeName = initNodeName
        if self.isDiscreteDistribution() == False:
            self.ParentsNum = len(initParents)
            if self.ProbabilityTable.shape[0]-2 != self.ParentsNum:
                raise NameError("Init failed, number of parents is wrong!")
        
    def getValueList(self):
        return list(set(self.ProbabilityTable[-2]))

    def isDiscreteDistribution(self):
        if self.ParentsList == None:
            return True
        else:
            False
    
    def getNodeProbability(self,ParentState):
        RunTable = self.ProbabilityTable
        for name in self.ParentsList:
            if name in ParentState:
                State = np.array(ParentState[name]).astype(dtype='<U21')
                index = np.where( RunTable[0] == State )[0]
                RunTable = RunTable[1:,index]  
            else:
                raise NameError("Parents state is not enough")
        return RunTable
    
    def getOutput(self,ParentState):
        if self.isDiscreteDistribution():   
            sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
        else:
            NodeProba = self.getNodeProbability(ParentState)
            sortedProba = NodeProba.T[NodeProba[1].argsort()].T            
        RandomValue = random.random()
        for index in range(len(sortedProba)):
            if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                ParentState[self.NodeName] = sortedProba[0,index]
                return ParentState
        ParentState[self.NodeName] = sortedProba[0,-1]
        return ParentState

        
    def getOutputWithLikehood(self, ParentState, ParentLikelihoodWeight, Observations):
        if self.NodeName not in Observations.keys():
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba = self.getNodeProbability(ParentState)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            RandomValue = random.random()
            for index in range(len(sortedProba)):
                if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                    ParentState[self.NodeName] = sortedProba[0,index]    
                    return ParentState, ParentLikelihoodWeight
            ParentState[self.NodeName] = sortedProba[0,-1]
            return ParentState, ParentLikelihoodWeight
        else:
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba = self.getNodeProbability(ParentState)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            index = np.where(sortedProba[0]==np.array(Observations[self.NodeName]).astype(dtype='<U21'))[0][0]
            ParentState[self.NodeName] = sortedProba[0,index]
            ParentLikelihoodWeight[self.NodeName] = float(sortedProba[1,index])
            return ParentState, ParentLikelihoodWeight

# Asia example

In [1619]:
asia = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'asia' )

In [1620]:
tuberculosis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.2 ],
     [ 'True', 'False', 0.8 ],
     [ 'False', 'True', 0.01 ],
     [ 'False', 'False', 0.99 ]], ['asia'],'tuberculosis')

In [1621]:
smoking = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'smoking' )

In [1622]:
lung = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.75 ],
     [ 'True', 'False', 0.25 ],
     [ 'False', 'True', 0.02 ],
     [ 'False', 'False', 0.98 ]], ['smoking'], 'lung' )

bronchitis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.92 ],
     [ 'True', 'False', 0.08 ],
     [ 'False', 'True', 0.03 ],
     [ 'False', 'False', 0.97 ]], ['smoking'], 'bronchitis' )

tuberculosis_or_cancer = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 1.0 ],
     [ 'True', 'True', 'False', 0.0 ],
     [ 'True', 'False', 'True', 1.0 ],
     [ 'True', 'False', 'False', 0.0 ],
     [ 'False', 'True', 'True', 1.0 ],
     [ 'False', 'True', 'False', 0.0 ],
     [ 'False', 'False', 'True', 0.0 ],
     [ 'False', 'False', 'False', 1.0 ]], ['tuberculosis', 'lung'], 'tuberculosis_or_cancer' )

In [1623]:
xray = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]], ['tuberculosis_or_cancer'] , 'xray')

In [1624]:
dyspnea = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 0.96 ],
     [ 'True', 'True', 'False', 0.04 ],
     [ 'True', 'False', 'True', 0.89 ],
     [ 'True', 'False', 'False', 0.11 ],
     [ 'False', 'True', 'True', 0.96 ],
     [ 'False', 'True', 'False', 0.04 ],
     [ 'False', 'False', 'True', 0.89 ],
     [ 'False', 'False', 'False', 0.11 ]], ['tuberculosis_or_cancer', 'bronchitis'],'dyspnea')

In [1625]:
NodeList = [asia,smoking,tuberculosis,lung,tuberculosis_or_cancer,bronchitis,xray,dyspnea]

# Normal Sampling

In [1626]:
RunForwardList = []
for i in range(300000):
    RunningVar = {}
    RunningVar = asia.getOutput(RunningVar)
    RunningVar = smoking.getOutput(RunningVar)
    RunningVar = tuberculosis.getOutput(RunningVar)
    RunningVar = lung.getOutput(RunningVar)
    RunningVar = tuberculosis_or_cancer.getOutput(RunningVar)
    RunningVar = bronchitis.getOutput(RunningVar)
    RunningVar = xray.getOutput(RunningVar)
    RunningVar = dyspnea.getOutput(RunningVar)
    RunForwardList.append(RunningVar)

In [1627]:
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }

In [1628]:
NodeNameList = ['asia','smoking','tuberculosis','lung','tuberculosis_or_cancer','bronchitis','xray','dyspnea']

In [1629]:
RunValueList = []
KeyList = RunForwardList[0].keys()
for RunForwardValue in RunForwardList:
    OneRow = []
    ObservationsCheck = True
    for key in Observations:
        if Observations[key] != RunForwardValue[key]:
            ObservationsCheck = False
    if ObservationsCheck == True:
        for key in RunForwardValue: 
            OneRow.append(RunForwardValue[key])
        RunValueList.append(OneRow)
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))

In [1630]:
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,len(np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0])/NumpyRunValue.shape[0])
    print('============================')

asia
True 0.9473684210526315
False 0.05263157894736842
smoking False
tuberculosis True
lung
True 0.02631578947368421
False 0.9736842105263158
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.9109311740890689
False 0.08906882591093117
dyspnea
True 0.951417004048583
False 0.048582995951417005


# Likelihood

In [1631]:
RunForwardList = []
RunForwardWeight = []
for i in range(100000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
    RunningVar,ParentLikelihoodWeight = asia.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = smoking.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = lung.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis_or_cancer.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = bronchitis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = xray.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = dyspnea.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)

In [1632]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [1633]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

asia
True 0.9521810079109848
False 0.04781899208901527
smoking False
tuberculosis True
lung
True 0.02044736666316764
False 0.9795526333368323
tuberculosis_or_cancer
True 1.0
False 0.0
bronchitis True
xray
True 0.8833640293536658
False 0.11663597064633414
dyspnea
True 0.9592932599173594
False 0.040706740082640684


# Monty Hall  Example 

In [1604]:
guest = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'guest' )
prize = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'prize' )

In [1605]:
monty = ConditionalProbabilityTable( 
    [[ 'A', 'A', 'A', 0.0 ],
     [ 'A', 'A', 'B', 0.5 ],
     [ 'A', 'A', 'C', 0.5 ],
     [ 'A', 'B', 'A', 0.0 ],
     [ 'A', 'B', 'B', 0.0 ],
     [ 'A', 'B', 'C', 1.0 ],
     [ 'A', 'C', 'A', 0.0 ],
     [ 'A', 'C', 'B', 1.0 ],
     [ 'A', 'C', 'C', 0.0 ],
     [ 'B', 'A', 'A', 0.0 ],
     [ 'B', 'A', 'B', 0.0 ],
     [ 'B', 'A', 'C', 1.0 ],
     [ 'B', 'B', 'A', 0.5 ],
     [ 'B', 'B', 'B', 0.0 ],
     [ 'B', 'B', 'C', 0.5 ],
     [ 'B', 'C', 'A', 1.0 ],
     [ 'B', 'C', 'B', 0.0 ],
     [ 'B', 'C', 'C', 0.0 ],
     [ 'C', 'A', 'A', 0.0 ],
     [ 'C', 'A', 'B', 1.0 ],
     [ 'C', 'A', 'C', 0.0 ],
     [ 'C', 'B', 'A', 1.0 ],
     [ 'C', 'B', 'B', 0.0 ],
     [ 'C', 'B', 'C', 0.0 ],
     [ 'C', 'C', 'A', 0.5 ],
     [ 'C', 'C', 'B', 0.5 ],
     [ 'C', 'C', 'C', 0.0 ]], ['guest','prize'], 'monty' )

In [1611]:
Observations = { 'guest' : 'A' }

In [1612]:
NodeList = [guest,prize,monty]

In [1616]:
RunForwardList = []
RunForwardWeight = []
for i in range(100000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    RunningVar,ParentLikelihoodWeight = guest.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = prize.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = monty.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)

In [1617]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [1618]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

guest A
prize
A 0.33427
C 0.33075000000000004
B 0.33498000000000006
monty
A 0.0
C 0.5030700000000001
B 0.49693000000000004
